In [1]:
from IPython.display import clear_output
import pandas as pd
import numpy as np
import requests

In [2]:
df = pd.read_csv("covid_19_04_04_2020.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 27 columns):
Unnamed: 0                                   4113 non-null int64
Patient Number                               3824 non-null float64
State Patient Number                         817 non-null object
Date Announced                               3490 non-null object
Age Bracket                                  798 non-null object
Gender                                       1027 non-null object
Detected City                                823 non-null object
Detected District                            2543 non-null object
Detected State                               3490 non-null object
State code                                   3490 non-null object
Current Status                               3490 non-null object
Notes                                        2557 non-null object
Contracted from which Patient (Suspected)    918 non-null object
Nationality                            

In [3]:
state_coordinate = pd.read_csv("state_lat_lon.csv")

In [33]:
filtered_data = []
for index, row in df.iterrows():
    temp = []
    temp.append('P'+str(row['Patient Number'])) #1
    temp.append(row['State Patient Number']) #2
    temp.append(row['Date Announced']) #3
    temp.append(row['Status Change Date']) #4
    temp.append(row['Age Bracket']) #5
    temp.append(row['Gender']) #6
    temp.append(row['Current Status']) #7
    temp.append(row['Nationality']) #8
    temp.append(row['Type of transmission']) #9
    temp.append(row['Contracted from which Patient (Suspected)']) #10
    temp.append(row['Detected State']) #11
    temp.append(row['Detected District']) #12
    temp.append(row['Detected City']) #13
    temp.append(get_from_loc(row['Notes'])) #14
    temp.append(row['Notes']) #15
    filtered_data.append(temp)
                

In [19]:
filtered_data[0:10]

[['P1.0',
  'KL-TS-P1',
  '30/01/2020',
  '14/02/2020',
  '30/01/2020',
  'F',
  'Recovered',
  'India',
  'Imported',
  nan,
  'Kerala',
  'Thrissur',
  'Thrissur',
  'WUHAN',
  'Travelled from Wuhan'],
 ['P2.0',
  'KL-AL-P1',
  '02/02/2020',
  '14/02/2020',
  '02/02/2020',
  nan,
  'Recovered',
  'India',
  'Imported',
  nan,
  'Kerala',
  'Alappuzha',
  'Alappuzha',
  'WUHAN',
  'Travelled from Wuhan'],
 ['P3.0',
  'KL-KS-P1',
  '03/02/2020',
  '14/02/2020',
  '03/02/2020',
  nan,
  'Recovered',
  'India',
  'Imported',
  nan,
  'Kerala',
  'Kasaragod',
  'Kasaragod',
  'WUHAN',
  'Travelled from Wuhan'],
 ['P4.0',
  'DL-P1',
  '02/03/2020',
  '15/03/2020',
  '02/03/2020',
  'M',
  'Recovered',
  'India',
  'Imported',
  nan,
  'Delhi',
  'East Delhi',
  'East Delhi (Mayur Vihar)',
  'AUSTRIA',
  'Travelled from Austria, Italy'],
 ['P5.0',
  'TS-P1',
  '02/03/2020',
  '02/03/2020',
  '02/03/2020',
  'M',
  'Recovered',
  'India',
  'Imported',
  nan,
  'Telangana',
  'Hyderabad',
  

In [34]:
df_filtered = pd.DataFrame(filtered_data, columns = ['Patient Number',
                                                    'State Patient Number',
                                                    'Date Announced',
                                                    'Status Change Date',
                                                    'Age Bracket',
                                                    'Gender',
                                                    'Current Status',
                                                    'Nationality',
                                                    'Type of transmission',
                                                    'Contracted from which Patient (Suspected)',
                                                    'Detected State',
                                                    'Detected District',
                                                    'Detected City',
                                                    'Travelled From which location',
                                                    'Notes'])
df_filtered.to_csv('covid_19_04_04_2020_filtered.csv')

In [15]:
def get_from_loc(s):
    if(pd.isna(s)):
        return 'NAN'
    import re
    pat1 = r'FROM\s[A-Z]+'
    pat2 = r'TO\s[A-Z]+'
    s = s.upper()
    f1 = re.search(pat1, s)
    f2 = re.search(pat2, s)
    r1 = ''
    r2 =''
    if(f1):
        data = (f1.group()).split(' ')[1]
        if(len(data) > 1):
            r1 = data
            return r1
    if(f2):
        data = (f2.group()).split(' ')[1]
        if(len(data) > 1):
            r2 = data
            return r2

In [16]:
get_from_loc('Travelled from Italy on March 8')

'ITALY'

In [4]:
app_id = "0INBjWlPQChfUSn13p4b"
app_code = "H9io3_9HVYbAv2nnhndrhQ"
url = "https://geocoder.api.here.com/6.2/geocode.json?searchtext={}&app_id={}&app_code={}"

In [5]:
cities = set(df['Detected City'].to_list())
city_to_state_map = {}
for each in zip(df['Detected City'].to_list(), df['Detected State'].to_list()):
    city_to_state_map[each[0]] = each[1]

In [6]:
try:
    cities.remove(' ')
    cities.remove(np.nan)
except:
    print("nothing to remove")

In [7]:
city_data = {}
failed_city = []
failed_url = []
index = 1
for city in cities:
    clear_output(wait = True)
    print(index, " of ", len(cities))
    index += 1
    r = requests.get(url.format(city +", "+ city_to_state_map[city] + ", india", app_id, app_code))
    try:
        raw_data = r.json()
        data = raw_data['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']
        city_data[city] = [data['Latitude'], data['Longitude']]
    except:
        print(r.json())
        failed_city.append(city)
        failed_url.append(url.format(city +", "+ city_to_state_map[city] + ", india", app_id, app_code))
        

233  of  233


In [22]:
city_data['Chuchot'] = [39.5976048,63.1020115]
city_data['Golekbana'] = [14.9837072,71.8195828]
city_data['Ladakh'] = [14.2605361,57.9144399]
city_data['Temple Gate'] = [10.4503473,73.7595634]
city_data['Chusot'] = [33.799711,77.7491035]
city_data['Mumbai Suburb'] = [19.1315563,72.7393767]
city_data['Gowribidanur'] = [13.6125592,77.4979894]

In [258]:
city_df = pd.DataFrame([[i,city_data[i][0], city_data[i][1]] for i in city_data], columns=['city_name', 'lat', 'lon'])

In [259]:
city_df.to_csv("city_lat_lon.csv")

In [ ]:
# now we need to find the places from whiche people travelled from


In [172]:
s1 = "Travelled from Wuhan".upper()
s2 = "Related to a previous patient from Sangli".upper()
s3 = "Evacuees from Iran (Jaisalmer army base camp)".upper()
s4 = "Travelled from Sharjah, travelled with another confimed case".upper()

In [177]:
pat = r'(FROM)\s[A-Z]+'
r = re.compile(pat)
print(re.search(pat, s2).group())
print(r.findall(s2))

FROM SANGLI
['FROM']


In [142]:
travel_from = df['Notes'].to_list()

In [183]:
import re
travel_from_country = {}
pat1 = r'FROM\s[A-Z]+'
pat2 = r'TO\s[A-Z]+'
for i in travel_from:
    if(not pd.isna(i)):
        s = i.upper()
        f1 = re.search(pat1, s)
        f2 = re.search(pat2, s)
        if(f1):
            data = (f1.group()).split(' ')[1]
            if(len(data) > 1):
                travel_from_country[data] = f1.group()
        if(f2):
            data = (f2.group()).split(' ')[1]
            if(len(data) > 1):
                travel_from_country[data] = f2.group()

In [247]:
travel_from_lat_lon = {}
index = 1
for i in travel_from_country:
    d = travel_from_country[i].split(' ')[1]
    clear_output(wait = True)
    print(index," of ",len(travel_from_country))
    index += 1
    r = requests.get(url.format(d, app_id, app_code))
    try:
        raw_data = r.json()
        data = raw_data['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']
        travel_from_lat_lon[i] = [data['Latitude'], data['Longitude']]
    except Exception as e:
        print(e)
        travel_from_lat_lon[i] = [np.nan]
    

97  of  100


ConnectionError: HTTPSConnectionPool(host='geocoder.api.here.com', port=443): Max retries exceeded with url: /6.2/geocode.json?searchtext=MIDDLEEAST&app_id=0INBjWlPQChfUSn13p4b&app_code=H9io3_9HVYbAv2nnhndrhQ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001F034BD3B38>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [254]:
travel_from_lat_lon

{'WUHAN': [30.64354, 114.32472],
 'AUSTRIA': [48.20263, 16.36843],
 'DUBAI': [25.26952, 55.30885],
 'BANGALORE': [12.96643, 77.58718],
 'ITALY': [41.90323, 12.49566],
 'IRAN': [35.68878, 51.41503],
 'THAILAND': [13.75337, 100.50483],
 'OMAN': [23.61524, 58.59126],
 'CHENNAI': [13.08363, 80.28252],
 'US': [38.89037, -77.03196],
 'MUMBAI': [18.94017, 72.83483],
 'PUNE': [18.50422, 73.85302],
 'JAPAN': [35.68409, 139.80885],
 'GREECE': [37.97614, 23.7364],
 'SAUDI': [22.27392, 46.71973],
 'QATAR': [25.29425, 51.51967],
 'FRANCE': [48.85718, 2.34141],
 'UK': [51.50643, -0.12721],
 'LONDON': [51.50643, -0.12721],
 'PHILIPPINES': [14.58865, 120.98454],
 'SPAIN': [40.41956, -3.69196],
 'RUSSIA': [55.75697, 37.61502],
 'NETHERLANDS': [52.36994, 4.90788],
 'KALYAN': [19.2395, 73.12979],
 'LISBON': [38.72639, -9.14949],
 'SCOTLAND': [55.95438, -3.20144],
 'SINGAPORE': [1.29041, 103.85211],
 'MALAYSIA': [3.1479, 101.69405],
 'UAE': [12.26908, -14.68863],
 'HYDERABAD': [17.39488, 78.47077],
 'DELH

In [255]:
travel_from_lat_lon['MIDDLE'] = [25.9409657,25.7593975]
travel_from_lat_lon['UNITED'] = [54.230956,-13.4363316]
travel_from_lat_lon['SRI'] = [7.8571784,78.4611594]
travel_from_lat_lon['WEST'] = [13.5571892,-87.967434]
travel_from_lat_lon['THIRUVANATHAPURAM'] = [8.5000475,76.7840779]

In [256]:
from_place_df = pd.DataFrame([[i, travel_from_country[i]] for i in travel_from_country], columns=['shortname', 'to_search'])
from_place_df.to_csv('from_place_data.csv')

In [257]:
from_place_df_latlon = pd.DataFrame([[i, travel_from_lat_lon[i][0], travel_from_lat_lon[i][1]] for i in travel_from_lat_lon], columns=['shortname', 'lat', 'lon'])
from_place_df_latlon.to_csv('from_place_lat_lon.csv')

In [246]:
travel_from_country

{'WUHAN': 'FROM WUHAN',
 'AUSTRIA': 'FROM AUSTRIA',
 'DUBAI': 'FROM DUBAI',
 'BANGALORE': 'TO BANGALORE',
 'ITALY': 'FROM ITALY',
 'IRAN': 'FROM IRAN',
 'THAILAND': 'FROM THAILAND',
 'OMAN': 'FROM OMAN',
 'CHENNAI': 'TO CHENNAI',
 'US': 'TO US',
 'MUMBAI': 'FROM MUMBAI',
 'PUNE': 'TO PUNE',
 'JAPAN': 'FROM JAPAN',
 'GREECE': 'FROM GREECE',
 'SAUDI': 'FROM SAUDI',
 'QATAR': 'FROM QATAR',
 'FRANCE': 'FROM FRANCE',
 'UK': 'FROM UK',
 'LONDON': 'FROM LONDON',
 'PHILIPPINES': 'FROM PHILIPPINES',
 'SPAIN': 'TO SPAIN',
 'RUSSIA': 'FROM RUSSIA',
 'NETHERLANDS': 'FROM NETHERLANDS',
 'KALYAN': 'FROM KALYAN',
 'LISBON': 'TO LISBON',
 'SCOTLAND': 'FROM SCOTLAND',
 'SINGAPORE': 'FROM SINGAPORE',
 'MALAYSIA': 'FROM MALAYSIA',
 'UAE': 'FROM UAE',
 'HYDERABAD': 'FROM HYDERABAD',
 'DELHI': 'TO DELHI',
 'INDIA': 'TO INDIA',
 'INDONESIA': 'TO INDONESIA',
 'TURKEY': 'FROM TURKEY',
 'DUBLIN': 'FROM DUBLIN',
 'GERMANY': 'TO GERMANY',
 'MECCA': 'FROM MECCA',
 'MEXICO': 'FROM MEXICO',
 'FINLAND': 'FROM FINLAN